In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import medfilt
from scipy.ndimage import gaussian_filter

In [ ]:
def d_arg(image_in, sz=1):
    #  image_filt = gaussian_filter(image_in, sigma=sz)
    image_in = medfilt(image_in, kernel_size=sz)
    d_arg = np.zeros(image_in.shape)

    for i in range(4):    
        # Rotate image in 90 deg increments, compute angle of gradient
        image_rot = np.rot90(image_in, i)
        grad_x,grad_y = np.gradient(image_rot)
        theta = np.arctan2(grad_y, grad_x)

        # Blur
        theta = gaussian_filter(theta, sigma=sz)
        
        # Compute change of angle of gradient in y direction (\alpha_{arg}, y_arg in direction 90*i)
        theta_grad_y = np.gradient(theta, axis=1)
        
        # Rotate back and add to total (accumulate darg)
        d_arg += np.rot90(theta_grad_y, -i)
        
        # Normalize and scale
        d_arg_2 = np.square(d_arg);
        d_arg_final = d_arg_2 / np.amax(d_arg_2) * 255
    
    return d_arg_final

In [ ]:
if __name__ == '__main__':
    
    # Load
    im = cv2.imread("../Images/Images_from_Liu_Bolin_s_site/Liu17.jpg")
    grayscale = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_out = d_arg(grayscale, 7).astype('uint8')
    
    # Threshold
    _, masks = cv2.threshold(im_out, 0.65 * 255, 255, cv2.THRESH_BINARY)
    im_final = cv2.cvtColor(masks, cv2.COLOR_GRAY2RGB)
    
    # Show output and overlay
    plt.figure(figsize=(15,15))
    plt.imshow(im_final)
    plt.figure(figsize=(15,15))
    out = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    out[im_final != 0] = 0
    im_final[:, :, 1:3] = 0
    plt.imshow(out / 255 * 0.4 + im_final / 255)